## Критерии Стьюдента

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import scipy
import statsmodels
from statsmodels.stats.weightstats import *
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**1.** Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?
Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

### Z-критерий
$$ Z(X^n) = \frac {\bar X - \mu_0 }{\sigma / \sqrt n} $$

### Гипотеза:
$$H_1: \mu \ne \mu_0 \Rightarrow p =2(1-F_{N(0,1)}(|z|))$$

In [2]:
X_mean = 9.57
mu_0 = 9.5
sigma = 0.4
n = 160
Z = (X_mean - mu_0) / (sigma / n**0.5)
p = 2 * (1 - scipy.stats.norm.cdf(Z))
print(f"p-value: {round(p, 4)}")

p-value: 0.0269


**Вывод:** Уровень значимости гипотезы $H_1$ о неравенстве средних < 0.05 $\rightarrow$ нулевая гипотеза о равенстве средниз отвергается на уровне значимости 0.05

**2.** Имеются данные о стоимости и размерах 53940 бриллиантов:

In [3]:
data = pd.read_csv('data\diamonds.txt', sep='\t')
data.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

* линейную регрессию с помощью LinearRegression без параметров 
* случайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости $\alpha=0.05$?

In [4]:
y = data.price
X = data.drop('price', axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [6]:
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators = 10, random_state=1)

In [7]:
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
rf_pred = rf.predict(X_test)

In [8]:
lr_pred_error = np.abs(y_test - lr_pred)
rf_pred_error = np.abs(y_test - rf_pred)

In [9]:
lfpe_mean = lr_pred_error.mean()
rfpe_mean = rf_pred_error.mean()
print(f"Среднее абсолютное отклонение для линейной регрессии: {lfpe_mean}\n\
для случайного леса: {rfpe_mean}")

Среднее абсолютное отклонение для линейной регрессии: 890.3764004285584
для случайного леса: 802.9205172724115


In [10]:
p_value = stats.ttest_rel(lr_pred_error, rf_pred_error).pvalue
print(f'p-value = {p_value}')

p-value = 1.6551745751479057e-38


**Вывод:** Уровень значимости гипотезы $H_1$ < 0.05 $\rightarrow$ нулевая гипотеза об одинаковом среднем качестве предсказаний отвергается

**3.** В предыдущей задаче посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до десятков (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).

In [11]:
print(f'95% confidence interval: {[*DescrStatsW(lr_pred_error - rf_pred_error).tconfint_mean()]}')

95% confidence interval: [74.28724532595253, 100.6245209863411]
